In [1]:
import numpy as np    
import cv2
import matplotlib.pyplot as pl   
import os 

def imshow(title = "", image = None, size =5):
    w= image.shape[0]
    h=image.shape[1]
    aspect_ratio = w/h     
    pl.figure(figsize = (size*aspect_ratio,size))
    pl.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    pl.title(title)
    pl.show()

In [4]:
def imshow(title = "", image = None, size = 5):
 
    pl.figure(figsize=(size , size))
    pl.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    pl.title(title)
    pl.show()

## Definition 1 for Background Subtraction Function 

In [4]:
def bgSubVid(video):
   
   cap = cv2.VideoCapture(video)

# Get the height and width of the frame (required to be an interfer)
   w = int(cap.get(3))
   h = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
   out = cv2.VideoWriter('name', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (w, h))

   kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
   fgbg = cv2.createBackgroundSubtractorKNN()

   while(1):
        ret, frame = cap.read()

        if ret:
          fgmask = fgbg.apply(frame)
          fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

          imshow('frame', fgmask)
        else:
           break  

   cap.release()
   out.release()


#bgSubVid(r"IMG_0845.mp4")

## Definition 2 for Background Subtraction Function 

In [13]:
def bgSubVid(video):
   
   cap = cv2.VideoCapture(video)

# Get the height and width of the frame (required to be an interfer)
   width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
   height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
   roi_size = (450, 1100)
   roi_center = (int(width/2), int(height/2))
   roi_tl = (int(roi_center[0] - roi_size[0]/12), int(roi_center[1] - roi_size[1]/4))


   kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
   fgbg = cv2.createBackgroundSubtractorKNN()

   while(1):
        ret, frame = cap.read()

        if ret:
          frame_roi = frame[roi_tl[1]:roi_tl[1]+roi_size[1], roi_tl[0]:roi_tl[0]+roi_size[0]]
          fgmask = fgbg.apply(frame_roi)
          fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

          imshow('frame', fgmask)
        else:
           break  

   cap.release()

### BG Subtraction/Segmentation:

1. Pre-processing of img 
2. Segmentation of objs 
3. Remove bg or apply filters 
4. Can be done by KNN, MOG, NNs, Img1 - Img2

### Image Subtraction:

1. Difference btw 2 frames 
2. Depends on speed of objs and frame rate 
3. Sensitive to threshold 
4. Sensitive to noise and moving objs 
5. Running average of bg

## Having the original color back on Foreground after Background Subtraction

In [34]:
def getBgFgSub(video):
    cap = cv2.VideoCapture(video)

    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    roi_size = (450, 1100)
    roi_center = (int(width/2), int(height/2))
    roi_tl = (int(roi_center[0] - roi_size[0]/12), int(roi_center[1] - roi_size[1]/4))


    # create a background subtractor object
    fgbg = cv2.createBackgroundSubtractorMOG2()

    # process each frame in the video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_roi = frame[roi_tl[1]:roi_tl[1]+roi_size[1], roi_tl[0]:roi_tl[0]+roi_size[0]]
        # apply background subtraction
        fgmask = fgbg.apply(frame_roi)
        
        # get original color on foreground
        color_fg = cv2.bitwise_and(frame_roi, frame_roi, mask=fgmask)
        
        # display the result
        cv2.imshow("Original Color on Foreground", color_fg)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
getBgFgSub('IMG_0845.mp4')

In [20]:
video = cv2.VideoCapture('IMG_0845.mp4')

fps = video.get(cv2.CAP_PROP_FPS)
print('frames per second =',fps)

minutes = 0
seconds = 20
frame_id = int(fps*(minutes*60 + seconds))
print('frame id =',frame_id)

frames per second = 60.0
frame id = 1200


In [8]:
cap = cv2.VideoCapture('IMG_0845.mp4')  # Replace 0 with the path to your video file if reading from file
bg_subtractor = cv2.createBackgroundSubtractorMOG2()
min_contour_area = 1000
max_contour_area = 5000

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale and perform background subtraction
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fg_mask = bg_subtractor.apply(gray)

    # Apply threshold and morphological operations
    thresh = cv2.threshold(fg_mask, 100, 255, cv2.THRESH_BINARY)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Find contours and filter out small and large contours
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_contour_area and area < max_contour_area:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display result
    cv2.imshow("Bottle Detection", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
